In [8]:
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)

True

In [2]:
api_key = os.getenv("CRUNCHBASE_API_KEY")
base_url = "https://api.crunchbase.com/api/"
params = {"user_key" : api_key}

In [4]:
query = {
    "field_ids": [
        "identifier",
        "location_identifiers",
        "short_description",
        "categories",
        "num_employees_enum",
        "revenue_range",
        "operating_status",
        "website",
        "linkedin"
    ],
    "limit": 1000,
    "query": [
        {
            "type": "predicate",
            "field_id": "location_identifiers",
            "operator_id": "includes",
            "values": [
                "4ce61f42-f6c4-e7ec-798d-44813b58856b" #UUID FOR LOS ANGELES
            ]
        },
        {
            "type": "predicate",
            "field_id": "facet_ids",
            "operator_id": "includes",
            "values": [
                "company"
        ]
        }
    ]
}

r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query, params=params)

In [9]:
json.loads(r.text)

{'count': 10110,
 'entities': [{'uuid': 'f8fb24b1-35e8-2c52-c4d6-53f2ce22292a',
   'properties': {'website': {'value': 'http://www.solofunds.com'},
    'identifier': {'permalink': 'solo-funds-inc',
     'image_id': 'ibh4nmusedlxisdm1olu',
     'uuid': 'f8fb24b1-35e8-2c52-c4d6-53f2ce22292a',
     'entity_def_id': 'organization',
     'value': 'SoLo Funds'},
    'linkedin': {'value': 'https://www.linkedin.com/company/solo-funds'},
    'short_description': 'SoLo is a lender company allowing members to easily access and supply short-term funds for immediate needs.',
    'revenue_range': 'r_00001000',
    'operating_status': 'active',
    'num_employees_enum': 'c_00011_00050',
    'categories': [{'entity_def_id': 'category',
      'permalink': 'financial-services',
      'uuid': '90b4194f-1d4f-ff5c-d7a6-6b6f32ae4892',
      'value': 'Financial Services'},
     {'entity_def_id': 'category',
      'permalink': 'fintech-e067',
      'uuid': 'e06799a9-f789-76e7-49a7-71ee980a70ec',
      'value'